# Compress frequency dependence of chi0

In [ ]:
using Revise
using Plots
using LinearAlgebra
using SparseIR
import SparseIR: valueim

newaxis = [CartesianIndex()]

In [ ]:
BLAS.set_num_threads(16)

In [ ]:
println(Threads.nthreads())

In [ ]:
using ITensors

println(ITensors.blas_get_num_threads())

In [ ]:
beta = 10.0

In [ ]:
# pole
pole = 1.0
calc_giv(v::FermionicFreq) = 1 / (valueim(v, beta) - pole)
calc_chi0(v::FermionicFreq, w::BosonicFreq) = calc_giv(v) * calc_giv(v + w)

In [ ]:
N = 8
half_N = N ÷ 2
nw = 2^N
half_nw = 2^(N-1)
nw

In [ ]:
# (fermionic freqs, bosonic freqs)
v = FermionicFreq.(2 .* collect(0:(nw-1)) .+ 1)
w = BosonicFreq.(2 .* collect((1-nw):0))
chi0 = calc_chi0.(v[:, newaxis], w[newaxis, :])

heatmap(abs.(chi0), c = :balance)

In [ ]:
# (f_1, ..., f_N, b_1, ..., b_N)
# The indices f_1 and b_1 correspond to the smallest scale in the momentum space.
# The indices f_N and b_N correspond to the largest scale in the momentum space.
tensor = reshape(chi0, repeat([2,], 2*N)...)
;

In [ ]:
# Rearrange the indices so that indices for larger length scales are to the right.
# ( (f_1, f_2), ...., (b_1, b_N))
dims = Int[]
for i in 1:N
    push!(dims, i)
    push!(dims, i+N)
end
println(dims)
tensor = reshape(permutedims(tensor, dims), repeat([4,], N)...)
;

In [ ]:
u, s, vt = svd(reshape(tensor, :, 4^half_N))
;

In [ ]:
plot(s./s[1], yaxis=:log, xlims=(0, 4^half_N))

In [ ]:
sites = siteinds(4, N)

In [ ]:
cutoff = 1E-10
maxdim = 100
M = MPS(tensor, sites; cutoff=cutoff, maxdim=maxdim)

In [ ]:
tensor_reconst = Array(reduce(*, M), sites...)
;

In [ ]:
plot(abs.(vec(tensor_reconst) .- vec(tensor)))

In [ ]:
bonddims = collect(size(m)[1] for m in M)
plot(bonddims, marker=:x)